In [37]:
import theano

import theano.scalar as ts
from theano.graph.basic import Apply, Constant, Variable
from theano.graph.op import COp, Op
from theano.misc.safe_asarray import _asarray
from theano.sparse.type import SparseType, _is_sparse
from theano.sparse import as_sparse_variable, structured_dot_grad

from theano.tensor.type import TensorType

from scipy.sparse import csr_matrix, csc_matrix

In [18]:
import string

import numpy as np
import pandas as pd

import sparsedot

# The "categorical variables"
strings = list(string.ascii_lowercase) + list(string.ascii_uppercase)
len(strings)

52

In [65]:
data = pd.DataFrame(
    {
        "y": np.random.normal(size=10000),
        "x": np.random.choice(strings, size=10000)
    }
)

X = np.asarray(pd.get_dummies(data["x"]))
X_sp = csr_matrix(X)

In [20]:
X_sp.format

'csr'

In [4]:
def tensor(*args, **kwargs):
    name = kwargs.pop("name", None)
    return TensorType(*args, **kwargs)(name=name)

In [5]:
#def structured_dot_grad(X, b, ga):
    # X: Sparse CSR
    # b: 1d dense
    
#    sdgcsx = sdg_csr
#    CSx = CSR

#g_A_data = sdgcsx(csm_indices(sparse_A), csm_indptr(sparse_A), dense_B, ga)
#    return CSx(g_A_data, csm_indices(sparse_A), csm_indptr(sparse_A), csm_shape(sparse_A))

In [71]:
class StructuredDot(Op):
    __props__ = ()

    def make_node(self, a, b):
        a = as_sparse_variable(a)
        assert a.format in ("csr", "csc", "bsr")

        dtype_out = ts.upcast(a.type.dtype, b.type.dtype)
        if b.type.ndim != 2:
            raise NotImplementedError("non-vector b")

        return Apply(self, [a, b], [tensor(dtype_out, (False, b.type.broadcastable[1]))])

    def perform(self, node, inputs, outputs):
        (X, b) = inputs
        (out,) = outputs
        
        
        if X.shape[1] != b.shape[0]:
            raise ValueError("shape mismatch in StructuredDot.perform", (X.shape, b.shape))

        # TODO: Necesitamos un algoritmo como 'matrix_vector' pero en C que calcule el 
        # producto entre matriz y vector en formato CSC
        if X.format == "csc":
            variable = X * b
        else: 
            offset = X.indptr
            column = X.indices
            value = X.data
            length = X.shape[0]
            result = np.zeros(length, dtype=b.dtype)
            variable = sparsedot.matrix_vector(offset, column, value, b.flatten(), result)[:, None]
       
    
        if isinstance(node.outputs[0].type, SparseType):
            assert _is_sparse(variable)
            out[0] = variable
            return

        # dot of an NxM sparse matrix, with a Mx1 dense matrix, returns vector
        # not matrix
        if variable.ndim == 1:
            variable = np.expand_dims(variable, 1)
        elif variable.ndim != 2:
            raise Exception("Output of structured dot should be a matrix " "(ndim=2)")

        assert variable.ndim == 2

        if variable.shape != (X.shape[0], b.shape[1]):
            if b.shape[0] == 1:
                raise Exception(
                    f"a.shape={X.shape}, b.shape={b.shape}, "
                    f"variable.shape={variable.shape}?  This is probably "
                    "because scipy.csc_matrix.dot has a bug "
                    "with singleton dimensions (i.e. "
                    "b.shape[0]=1) in SciPy 0.6.  Use SciPy "
                    f"0.7.  (You have SciPy version)"
                )
            else:
                raise Exception(
                    f"a.shape={X.shape}, b.shape={b.shape}, variable.shape={variable.shape}?"
                )
       
        out[0] = _asarray(variable, str(variable.dtype))
        
        return None
        
    def grad(self, inputs, gout):
        # a is sparse, b is dense, g_out is dense
        # ga = g_out x b.T
        # gb = a.T x g_out
        (X, b) = inputs
        (g_out,) = gout
        return [structured_dot_grad(X, b, g_out), structured_dot(X.T, g_out)]

    def infer_shape(self, fgraph, node, shapes):
        return [(shapes[0][0], shapes[1][1])]

structured_dot = StructuredDot()

In [72]:
import pymc3 as pm

In [73]:
with pm.Model() as model:
    b = pm.Normal("b", shape=52)
    sigma = pm.Exponential("sigma", lam=1)
    mu = theano.sparse.structured_dot(csr_matrix(X), b[:, None]).flatten()
    pm.Normal("y", mu=mu, sigma=sigma, observed=data["y"])
    
with model:
    idata = pm.sample(return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, b]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.


In [74]:
with pm.Model() as model:
    b = pm.Normal("b", shape=52)
    sigma = pm.Exponential("sigma", lam=1)
    mu = structured_dot(csr_matrix(X), b[:, None]).flatten()
    pm.Normal("y", mu=mu, sigma=sigma, observed=data["y"])
    
with model:
    idata = pm.sample(return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [sigma, b]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 6 seconds.
